# Evaluate Model

This notebook compares the performance of the trained ActiNet model to a balanced random forest model, similar to the pypi:accelerometer model.

In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt

from actinet.models import ActivityClassifier, RFActivityClassifier
from actinet.prepare import load_all_and_make_windows, extract_accelerometer_features, \
    prepare_accelerometer_data
from actinet.evaluate import evaluate_models
from actinet.utils.eval_utils import *

WINSEC = 30 # seconds
SAMPLE_RATE = 100 # Hz
RESAMPLE_RATE = 30 # Hz
N_JOBS = 8 # Set to higher number for quicker execution, but don't exceed max.

DATAFILES = f"data/capture24/P[0-9][0-9][0-9].csv.gz"
ANNOFILE = f"data/capture24/annotation-label-dictionary.csv"
SAVEFOLDER = f"data/capture24"

ACTIVITY_LABELS = ["light", "moderate-vigorous", "sedentary", "sleep"]

## Evaluate actinet against accelerometer

First we extract the features each of the capture 24 files using the accelerometer package

In [ ]:
if len(glob("data/capture24/bbaa/P[0-9][0-9][0-9]-epoch.csv.gz")) != 151:
    extract_accelerometer_features(n_jobs=N_JOBS)

Next we prepare the participant accelerometer data into the expected shape, containing the X, Y, T and P

In [ ]:
# Accelerometer feature data prepared
if len(glob(f"{SAVEFOLDER}/prepared/accelerometer/*.npy")) == 4:
    X_bbaa = np.load(f"{SAVEFOLDER}/prepared/accelerometer/X.npy")
    Y_bbaa = np.load(f"{SAVEFOLDER}/prepared/accelerometer/Y.npy")
    T_bbaa = np.load(f"{SAVEFOLDER}/prepared/accelerometer/T.npy")
    P_bbaa = np.load(f"{SAVEFOLDER}/prepared/accelerometer/pid.npy")

else:
    X_bbaa, Y_bbaa, T_bbaa, P_bbaa = prepare_accelerometer_data(ANNOFILE, SAVEFOLDER, N_JOBS)

In [ ]:
# Actinet data prepared
if len(glob(f"{SAVEFOLDER}/prepared/downsampling_linear_lowpass_None/*.npy")) == 4:
    X_actinet = np.load(f"{SAVEFOLDER}/prepared/downsampling_linear_lowpass_None/X.npy")
    Y_actinet = np.load(f"{SAVEFOLDER}/prepared/downsampling_linear_lowpass_None/Y.npy")
    T_actinet = np.load(f"{SAVEFOLDER}/prepared/downsampling_linear_lowpass_None/T.npy")
    P_actinet = np.load(f"{SAVEFOLDER}/prepared/downsampling_linear_lowpass_None/pid.npy")

else:
    X_actinet, Y_actinet, T_actinet, P_actinet = load_all_and_make_windows(
        datafiles=glob(DATAFILES), 
        annofile=ANNOFILE, 
        out_dir=SAVEFOLDER, 
        anno_label="Walmsley2020",
        sample_rate=SAMPLE_RATE,
        winsec=WINSEC,
        n_jobs=N_JOBS,
        downsampling_method="linear",
        lowpass_hz=None,
        resample_rate=RESAMPLE_RATE,
    )

Evaluate model using 5 fold stratified group cross validation

In [ ]:
actinet_res_path = "outputs/actinet_vs_bbaa/actinet_results.pkl"
bbaa_res_path = "outputs/actinet_vs_bbaa/rf_results.pkl"

if not os.path.exists(actinet_res_path) or not os.path.exists(bbaa_res_path):
    bbaa_classifier = RFActivityClassifier(
        winsec=WINSEC,
        labels=ACTIVITY_LABELS,
        n_estimators=1000,
        oob_score=True,
        sampling_strategy="not minority",
        replacement=True,
        n_jobs=N_JOBS,
        random_state=42,
        verbose=1
    )

    actinet_classifier = ActivityClassifier(
        labels = np.unique(Y_actinet),
        batch_size=1000,
        device="cuda:0",
        verbose=True
    )

    res = evaluate_models(
        actinet_classifier,
        bbaa_classifier,
        X_actinet,
        X_bbaa,
        Y_actinet,
        Y_bbaa,
        P_actinet,
        P_bbaa,
        T_actinet,
        T_bbaa,
        weights_path="models/evaluation_models/actinet_vs_bbaa_{}.pt",
        out_dir="outputs/actinet_vs_bbaa",
        verbose=True,
    )

results_bbaa = pd.read_pickle(bbaa_res_path)
results_actinet = pd.read_pickle(actinet_res_path)

In [ ]:
fold_pid_df = pd.DataFrame([{'Fold': fold+1, "Test Participant IDs": ", ".join(sorted(set(group)))} for fold, group in results_actinet["group"].items()]).set_index("Fold")
fold_pid_df.to_csv("outputs/actinet_vs_bbaa/fold_pids.csv")
fold_pid_df

In [ ]:
data = {
    'accelerometer': {'y': np.hstack(results_bbaa["Y_true"]), 
                      'y_pred': np.hstack(results_bbaa["Y_pred"]), 
                      'pid': np.hstack(results_bbaa["group"])
                      },
    'actinet': {'y': np.hstack(results_actinet["Y_true"]), 
                'y_pred': np.hstack(results_actinet["Y_pred"]), 
                'pid': np.hstack(results_actinet["group"])
                }
}

In [ ]:
results = []

for model, model_data in data.items():
    for pid in np.unique(model_data['pid']):
        mask = model_data['pid'] == pid
        y_true = model_data['y'][mask]
        y_pred = model_data['y_pred'][mask]
        accuracy, f1, kappa, bacc = calculate_metrics(y_true, y_pred)
        results.append({'Participant': pid, 'Model': model, "Balanced Accuracy": bacc,
                        'Accuracy': accuracy, 'Macro F1': f1, 'Cohen Kappa': kappa,
                        'Predicted': y_pred, 'True': y_true, "Pred_dict": DivDict(pd.value_counts(y_pred).to_dict())/120,
                        "True_dict": DivDict(pd.value_counts(y_true).to_dict())/120, 
                        "Len": len(y_true)})

results = pd.DataFrame(results)

In [ ]:
# Group by model and calculate mean and standard deviation
summary = results.groupby('Model')[['Accuracy',
                                    'Balanced Accuracy',
                                    'Cohen Kappa',
                                    'Macro F1']].agg(lambda x: f"{np.mean(x):.3f} " + 
                                                                  f"± {np.std(x):.3f}")

summary

In [ ]:
metadata = pd.read_csv("data/capture24/metadata.csv")
results_df = results.merge(metadata, left_on="Participant", right_on="pid")

sex_mapping = {'F': 'Female', 'M': 'Male'}
results_df['Sex'] = pd.Categorical(results_df['sex'].map(sex_mapping), ordered=True,
                                   categories=sex_mapping.values())

results_df['Age Band'] = pd.Categorical(results_df['age'], ordered=True,
                                        categories=['18-29', '30-37', '38-52', '53+'])

results_df.drop(columns=["age", "sex", "pid"], inplace=True)

Paired t-test

In [ ]:
from scipy.stats import ttest_rel

metrics = ['Accuracy', 'Balanced Accuracy', 'Cohen Kappa', 'Macro F1']
p_values = {}

for metric in metrics:
    acc_values = results_df.loc[results_df['Model'] == 'accelerometer', metric].values
    actinet_values = results_df.loc[results_df['Model'] == 'actinet', metric].values
    _, p_value = ttest_rel(acc_values, actinet_values)
    p_values[metric] = p_value

p_values

Difference boxplots

In [ ]:
plot_difference_boxplots(results_df)

In [ ]:
plot_model_performance(results, 'Accuracy', modulus=10)

In [ ]:
plot_model_performance(results, 'Macro F1', 10)

In [ ]:
plot_model_performance(results, 'Cohen Kappa', 10)

In [ ]:
def plot_boxplots(df, x, y='Macro F1', hue='Model'):
    """Plots boxplots of model performance by a specified variable"""
    _, ax = plt.subplots(figsize=(5, 3), dpi=1000)
    with sns.color_palette("Set1"):
        sns.boxplot(data=df, x=x, y=y, hue=hue, ax=ax)
    ax.set_xlabel("Age Band")
    ax.set_ylabel(f"Cohen's kappa score")
    plt.title(f"Performance by {x}")
    plt.show()

In [ ]:
plot_boxplots(results_df, 'Age Band', y="Cohen Kappa")

In [ ]:
plot_boxplots(results_df, 'Sex',  y="Cohen Kappa")

In [ ]:
# Group by model and calculate mean and standard deviation
results_df.groupby(['Age Band', 'Model'])[['Cohen Kappa']].agg(lambda x: f"{np.mean(x):.3f} " + 
                                                                  f"± {np.std(x):.3f}")

In [ ]:
# Group by model and calculate mean and standard deviation
results_df.groupby(['Sex', 'Model'])[['Cohen Kappa']].agg(lambda x: f"{np.mean(x):.3f} " + 
                                                                  f"± {np.std(x):.3f}")

In [ ]:
results_df.head()

Confusion matrices

In [ ]:
def generate_confusion_matrices(results_df, group_by=None, save_path=None, fontsize=20):
    """Generates and plots confusion matrices for different subgroups.""" 
    if group_by is None:  # Full population
        fig, axs = plt.subplots(1, 2, sharey=True, figsize=(12, 6), dpi=800)
        fig.suptitle("Confusion matrices for full Capture-24 population\nusing 5-fold group cross-validation", 
                     fontsize=fontsize)
        
        y_true_bbaa, y_pred_bbaa, y_true_actinet, y_pred_actinet, _ = build_confusion_matrix_data(results_df)
        
        plot_confusion_matrix(y_true_bbaa, y_pred_bbaa, 'accelerometer', ax=axs[0], fontsize=fontsize*0.8)
        plot_confusion_matrix(y_true_actinet, y_pred_actinet, 'actinet', ax=axs[1], fontsize=fontsize*0.8)
        fig.tight_layout()
        plot_and_save_fig(fig, save_path)
    
    else:
        unique_groups = results_df[group_by].cat.categories
        fig = plt.figure(figsize=(12, 6*len(unique_groups)), dpi=800, constrained_layout=True)
        fig.suptitle(f"Confusion matrices for different {group_by.lower()} in " +
                     "Capture-24 population\nusing 5-fold group cross-validation", fontsize=fontsize)

        subfigs = fig.subfigures(nrows=len(unique_groups), ncols=1)
        
        for group, subfig in zip(unique_groups, subfigs):
            y_true_bbaa, y_pred_bbaa, y_true_actinet, y_pred_actinet, population =\
                build_confusion_matrix_data(results_df, **{group_by.replace(' ', '_').lower(): group})

            subfig.suptitle(f"{group_by}: {group} (n = {population})", fontsize=fontsize*0.9)
            axs = subfig.subplots(nrows=1, ncols=2, sharey=True)

            plot_confusion_matrix(y_true_bbaa, y_pred_bbaa, 'accelerometer', ax=axs[0], fontsize=fontsize*0.8)
            plot_confusion_matrix(y_true_actinet, y_pred_actinet, 'actinet', ax=axs[1], fontsize=fontsize*0.8)
        
        plot_and_save_fig(fig, save_path)

In [ ]:
generate_confusion_matrices(results_df, save_path="outputs/actinet_vs_bbaa/full_population.pdf", fontsize=18)
generate_confusion_matrices(results_df, group_by="Sex", save_path="outputs/actinet_vs_bbaa/by_sex.pdf", fontsize=18)
generate_confusion_matrices(results_df, group_by="Age Band", save_path="outputs/actinet_vs_bbaa/by_age.pdf", fontsize=18)

Bland-Altman plots

In [ ]:
generate_bland_altman_plots(results_df, save_path="outputs/actinet_vs_bbaa/bland_altman/full_population.pdf")
generate_bland_altman_plots(results_df[results_df["Sex"]=="Female"], subset="female",
                            save_path="outputs/actinet_vs_bbaa/bland_altman/by_sex_female.pdf")
generate_bland_altman_plots(results_df[results_df["Sex"]=="Male"], subset="male",
                            save_path="outputs/actinet_vs_bbaa/bland_altman/by_sex_male.pdf")
generate_bland_altman_plots(results_df, group_by="Age Band", save_path="outputs/actinet_vs_bbaa/bland_altman/by_age.pdf")